In [1]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import time
from sklearn import *

from itertools import product
# from sklearn import tree
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix,accuracy_score
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.linear_model import LogisticRegression



In [2]:
#Load Data into Dataframe for Stacking 
# Load the training data
df_X = pd.read_table('data/X_train.txt', delim_whitespace=True, names=None,header=None)
df_Y = pd.read_table('data/Y_train.txt', delim_whitespace=True, names=None,header=None)

# And the test features
df_Xte = pd.read_table('data/X_test.txt', delim_whitespace=True, names=None,header=None)


In [3]:
#Check length of test data
df_Xte.shape[0]

200000

In [4]:
#Check if data has been loaded correctly 
df_X.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,242.0,227.0,240.92,232.44,1195.0,253.0,0.0,1.26710,6.4128,1.98690,3.9756,2.3392,6.3537,0.0
1,249.0,230.0,242.31,233.68,1579.0,243.0,0.0,9.68310,6.0824,1.19640,3.4577,2.0416,7.6746,0.0
2,223.0,195.0,227.64,204.42,1034.0,603.0,318.0,1.52860,17.8690,13.23000,5.7120,4.7216,6.6030,0.0
3,234.0,221.0,236.27,229.73,7716.0,3907.0,0.0,0.60465,8.0497,3.44760,2.4845,1.5741,1.4205,0.0
4,234.0,233.0,245.51,234.10,545.0,21.0,0.0,6.74730,5.2649,0.86766,5.9626,4.2450,3.1429,24.2


In [5]:
#Check if data has been loaded correctly 
df_Y.head()

,0
0,0
1,1
2,1
3,1
4,0


In [6]:
#Check Test data load
df_Xte.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,246.00,226.0,243.27,233.43,7121.0,648.0,0.0,1.1235,5.5226,1.9747,2.3390,1.1438,5.3022,0.0
1,239.00,226.0,240.77,233.31,2109.0,560.0,0.0,5.7154,6.3213,1.5999,4.3318,3.1834,3.0579,0.0
2,251.62,232.0,245.82,233.57,199.0,14.0,0.0,3.4131,5.7395,1.2936,5.6142,2.0601,20.0000,0.0
3,253.00,243.0,249.18,249.18,4095.0,0.0,0.0,10.3200,2.5050,0.0000,1.5908,1.2399,20.0000,0.0
4,247.00,240.0,248.12,248.12,321.0,0.0,0.0,10.8170,3.4157,0.0000,4.2366,1.2236,20.0000,0.0


In [7]:
# Load Data in the form of arrays to perform individual Model training & testing
# Load the training data
X = np.genfromtxt('data/X_train.txt', delimiter=None)
Y = np.genfromtxt('data/Y_train.txt', delimiter=None)

# And the test features
Xte = np.genfromtxt('data/X_test.txt', delimiter=None)

In [8]:
#Check Shape
print(X.shape, Y.shape, Xte.shape)

(200000, 14) (200000,) (200000, 14)


In [9]:
#Check loaded test data
Xte

array([[246.    , 226.    , 243.27  , ...,   1.1438,   5.3022,   0.    ],
       [239.    , 226.    , 240.77  , ...,   3.1834,   3.0579,   0.    ],
       [251.62  , 232.    , 245.82  , ...,   2.0601,  20.    ,   0.    ],
       ...,
       [252.    , 237.    , 248.23  , ...,   1.8303,  20.    ,   0.    ],
       [247.    , 235.    , 247.1   , ...,   2.1439,  20.    ,   0.    ],
       [247.    , 238.    , 248.61  , ...,   1.5008,  20.    ,   0.    ]])

In [10]:
#Check X data
X

array([[242.    , 227.    , 240.92  , ...,   2.3392,   6.3537,   0.    ],
       [249.    , 230.    , 242.31  , ...,   2.0416,   7.6746,   0.    ],
       [223.    , 195.    , 227.64  , ...,   4.7216,   6.603 ,   0.    ],
       ...,
       [250.    , 234.    , 244.51  , ...,   3.3698,  20.    ,   0.    ],
       [236.5   , 227.    , 241.98  , ...,   2.7834,   2.5209,   0.    ],
       [242.    , 229.    , 241.38  , ...,   1.9358,   2.1394,   0.    ]])

In [11]:
#Check Y data
Y

array([0., 1., 1., ..., 0., 0., 1.])

In [12]:
# split Array data training / validation data for individual model training
x_train, x_validation, y_train, y_validation = model_selection.train_test_split(
    X, Y, test_size=0.2, random_state=42)

In [13]:
y_validation

array([0., 0., 1., ..., 0., 0., 0.])

### Functions STARTS

In [26]:
#Function to save Individual Model Test Probabilities for Submission to Kaggle
def SavetoTextFile(model_name,probs,Xte):
    # Create the data for submission by taking the P(Y=1) column from probs and just add a running index as the first column.
    Y_sub = np.vstack([np.arange(Xte.shape[0]), probs[:, 1]]).T
    # We specify the header (ID, Prob1) and also specify the comments as '' so the header won't be commented out with
    # the sign.
    file_name='Solutions/Y_sub_'+model_name+'.txt'
    np.savetxt(file_name, Y_sub, '%d, %.5f', header='ID,Prob1', comments='', delimiter=',')
    print('{} Saved'.format(file_name))
    

In [42]:
def auc_calculate_print(x,y,classifier):
    roc = metrics.roc_auc_score(y,classifier.predict_proba(x)[:,1])
    return(roc)

### Functions ENDS

### Random Forest STARTS

Hyper Paramters of Random Forest:

- n_estimators --> No of trees
- criterion --> entropy or gini
- max_features --> Use Sub set of list of features
- bootstrap--> True by default
- oob_score --> default = False 

In [14]:
#Execute this

rfc=RandomForestClassifier(n_estimators=65, criterion = 'entropy', random_state = 0,n_jobs=-1, oob_score=True)

print("Random Forest training started")

starting_time = time.time()

rfc.fit(x_train, y_train)

end_time = time.time()

print("Random Forest training finished, took {} seconds".format(end_time - starting_time))

print('Validation AUC: {}'.format(metrics.roc_auc_score(y_validation, rfc.predict_proba(x_validation)[:,1])))

print('Train AUC: {}'.format(metrics.roc_auc_score(y_train, rfc.predict_proba(x_train)[:,1])))

Random orest training started
Random Forest training finished, took 75.52354693412781 seconds
Validation AUC: 0.7598094409091216
Train AUC: 0.9914036951932823


In [17]:
#Getting the individual Probabilities
#This is what needs to be converted to txt for submission

rfc_probs = rfc.predict_proba(Xte)
print(rfc_probs[:5])

[[0.         1.        ]
 [0.50923077 0.49076923]
 [0.75692308 0.24307692]
 [0.96879121 0.03120879]
 [0.84615385 0.15384615]]


In [18]:
#Check if no. of rows of predicted probabilites & Xtest match
print(Xte.shape[0],rfc_probs.shape[0])

200000 200000


In [29]:
# Make the prediction probabilities text file for Submission to Kaggle
SavetoTextFile('rfc',rfc_probs,Xte)

Solutions/Y_sub_rfc.txt Saved


In [15]:
#Out of Bag Score
#Only relevent to Random Forest
print('OOB Score: {}'.format(rfc.oob_score_))
print('Score: {} '.format(rfc.score(x_validation, y_validation)))

OOB Score: 0.72418125
Score: 0.72515 


*** Random Forest Hyper Parameter Optimization ***

RandomForest Classifier Choosing the Optimial model
- n_estimators=65, criterion = 'entropy',n_jobs=-1, oob_score=True  - 0.7598 Validation AUC, .9914 Train AUC - 47 secs
- n_estimators=150, criterion = 'entropy',n_jobs=-1, oob_score=True - 0.7609 Validation AUC, .9918 Train AUC - 112 secs
- n_estimators=200, criterion = 'entropy',n_jobs=-1, oob_score=True - 0.7613 Validation AUC, .9919 Train AUC - 168 secs
- n_estimators=400, criterion = 'entropy',n_jobs=-1, oob_score=True - 0.7615 Validation AUC, .9919 Train AUC - 371 secs

No Significant change in Validation accuracy with increase in no. of trees so changing criterion & trying

- n_estimators=400, criterion = 'gini',n_jobs=-1, oob_score=True - 0.7605 Validation AUC, .9920 Train AUC - 371 secs

No Significant change in Validation accuracy with change in Criterion = Gini
Setting oob_score=False & Trying

- n_estimators=400, criterion = 'gini',n_jobs=-1, oob_score=False - 0.7605 Validation AUC, .9920 Train AUC - 215 secs


*** Conclusion: ***

From the above results we choose the below as our optimal Hyper parameters:

- n_estimators=65,criterion = 'entropy',n_jobs=-1, oob_score=True
- The above gives us a Validation AUC of 0.7598 & takes only 47 seconds to train
- Criterion : Gini & Entroy : These usually yield similar results so we can choose either one, although Gini is a little Faster

 



### Random FOrest ENDS

### Naive Bayes STARTS

In [25]:
naive_bayes_classifier = GaussianNB()

print("Naive Bayes training started")

starting_time = time.time()

naive_bayes_classifier.fit(x_train, y_train)

end_time = time.time()

print("Naive Bayes training finished, took {} seconds".format(end_time - starting_time))

#Validation AUC Calculation

#Since roc_auc_score needs probabilies as input we r using .predict_proba
naive_bayes_classifier_roc_validation = metrics.roc_auc_score(
    y_validation, naive_bayes_classifier.predict_proba(x_validation)[:,1])

print('Naive Bayes Validation AUC: {}'.format(naive_bayes_classifier_roc_validation))

#Training AUC Calculation
naive_bayes_classifier_roc_train = metrics.roc_auc_score(
   y_train, naive_bayes_classifier.predict_proba(x_train)[:,1])
print('Naive Bayes Train AUC: {}'.format(naive_bayes_classifier_roc_train))

Naive Bayes training started
Naive Bayes training finished, took 0.22266769409179688 seconds
Naive Bayes Validation AUC: 0.6124095289184106
Naive Bayes Train AUC: 0.6124943821019428


In [27]:
#Getting the individual Probabilities
#This is what needs to be converted to txt for submission

naive_bayes_classifier_probs = naive_bayes_classifier.predict_proba(Xte)
print(naive_bayes_classifier_probs[:5])

[[0.59620622 0.40379378]
 [0.57795356 0.42204644]
 [0.80870104 0.19129896]
 [0.98150997 0.01849003]
 [0.97542329 0.02457671]]


In [28]:
# Make the prediction probabilities text file for Submission to Kaggle
SavetoTextFile('naive_bayes',naive_bayes_classifier_probs,Xte)

Solutions/Y_sub_naive_bayes.txt Saved


In [30]:
#Cross Checking scores
#For reference only
y_pred = naive_bayes_classifier.predict(x_validation)

In [31]:
#Cross Checking scores
#For reference only
cm = confusion_matrix(y_validation,y_pred)
print(cm)
accuracy_score(y_validation,y_pred)

[[17409  8882]
 [ 6655  7054]]


0.611575

### Naive Bayes ENDS

### Decision Tree STARTS

In [36]:
#Execute
decision_tree_classifier = tree.DecisionTreeClassifier(random_state=0)

print("Decision Tree training started")

starting_time = time.time()

decision_tree_classifier.fit(x_train, y_train)

end_time = time.time()

print("Decision Tree training finished, took {} seconds".format(end_time - starting_time))

#Validation AUC Calculation

decision_tree_classifier_validation_roc = auc_calculate_print(x_validation,y_validation,decision_tree_classifier)

print('Decision Tree Validation AUC: {}'.format(decision_tree_classifier_validation_roc))

#Training AUC Calculation

decision_tree_classifier_train_roc= auc_calculate_print(x_train,y_train,decision_tree_classifier)

print('Decision Tree Train AUC: {}'.format(decision_tree_classifier_train_roc))



Decision Tree training started
Decision Tree training finished, took 5.874881029129028 seconds
Decision Tree Validation AUC: 0.6599961391510298
Decision Tree Train AUC: 0.9937726770838594


In [37]:
#Getting the individual Probabilities
#This is what needs to be converted to txt for submission

decision_tree_classifier_probs = decision_tree_classifier.predict_proba(Xte)
print(decision_tree_classifier_probs[:5])

[[0. 1.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]]


In [38]:
# Make the prediction probabilities text file for Submission to Kaggle
SavetoTextFile('decision_tree',decision_tree_classifier_probs,Xte)

Solutions/Y_sub_decision_tree.txt Saved


In [39]:
#Cross Checking scores
#For reference only
y_pred = decision_tree_classifier.predict(x_validation)

In [40]:
#Cross Checking scores
#For reference only
cm = confusion_matrix(y_validation,y_pred)
print(cm)
accuracy_score(y_validation,y_pred)

[[20363  5928]
 [ 6658  7051]]


0.68535

### Decision Tree ENDS

### Ada Boost STARTS

learning_ratefloat, default=1.
- Learning rate shrinks the contribution of each classifier by learning_rate. There is a trade-off between learning_rate and n_estimators.


In [41]:
ada_boost_classifier = AdaBoostClassifier(tree.DecisionTreeClassifier(random_state=0),
                                          algorithm='SAMME.R',n_estimators=200, learning_rate=0.5)

print("Ada Boost training started")

starting_time = time.time()

ada_boost_classifier.fit(x_train, y_train)

end_time = time.time()

print("Ada Boost training finished, took {} seconds".format(end_time - starting_time))

#Validation AUC Calculation

ada_boost_classifier_validation_roc = auc_calculate_print(x_validation,y_validation,ada_boost_classifier)

print('Ada Boost Validation AUC: {}'.format(ada_boost_classifier_validation_roc))

#Training AUC Calculation

ada_boost_classifier_train_roc= auc_calculate_print(x_train,y_train,ada_boost_classifier)

print('Ada Boost Train AUC: {}'.format(ada_boost_classifier_train_roc))

Ada Boost training started
Ada Boost training finished, took 915.8517315387726 seconds
Ada Boost Validation AUC: 0.6599961391510298
Ada Boost Train AUC: 0.9937726770838594


In [44]:
#Getting the individual Probabilities
#This is what needs to be converted to txt for submission

ada_boost_classifier_probs = ada_boost_classifier.predict_proba(Xte)
print(ada_boost_classifier_probs[:5])

[[3.11943906e-03 9.96880561e-01]
 [1.56564617e-01 8.43435383e-01]
 [9.84272850e-01 1.57271498e-02]
 [9.99869853e-01 1.30146850e-04]
 [9.99409280e-01 5.90720119e-04]]


In [45]:
# Make the prediction probabilities text file for Submission to Kaggle
SavetoTextFile('ada_boost',ada_boost_classifier_probs,Xte)

Solutions/Y_sub_ada_boost.txt Saved


In [46]:
#Cross Checking scores
#For reference only
y_pred = ada_boost_classifier.predict(x_validation)

In [47]:
#Cross Checking scores
#For reference only
cm = confusion_matrix(y_validation,y_pred)
print(cm)
accuracy_score(y_validation,y_pred)

[[21354  4937]
 [ 6138  7571]]


0.723125

*** Ada Boost Hyper Parameter Optimization ***

*** n_estimators: ***

- DecisionTreeClassifier(random_state=0),algorithm='SAMME.R',n_estimators=100, learning_rate=0.5  - 0.7277 Validation AUC,  .9937 Train AUC
- DecisionTreeClassifier(random_state=0),algorithm='SAMME.R',n_estimators=200, learning_rate=0.5  - 0.7374 Validation AUC,  .9937 Train AUC - 489 Secs
- DecisionTreeClassifier(random_state=0),algorithm='SAMME.R',n_estimators=500, learning_rate=0.5  - 0.7476 Validation AUC,  .9937 Train AUC - 1067 Secs

Since, the Validation AUC is not improving substantially .7274 (n_estimators=200) Vs 0.7476 (n_estimators=500) we change the learning rate to 0.7 & n_estimators=200

*** learning_rate: ***

- DecisionTreeClassifier(random_state=0),algorithm='SAMME.R',n_estimators=500, learning_rate=0.7  - 0.7390 Validation AUC,  .9937 Train AUC - 1067 Secs

Since, learning rate = 0.7 caused a drop in the AUC accuracy from .7476 (learning_rate=0.5) to  0.7390 (learning_rate=0.7) we decrease the learning rate to 0.6 & check the results

- DecisionTreeClassifier(random_state=0),algorithm='SAMME.R',n_estimators=200, learning_rate=0.6  - 0.7387 Validation AUC,  .9937 Train AUC - 459 Secs
- DecisionTreeClassifier(random_state=0),algorithm='SAMME.R',n_estimators=500, learning_rate=0.6  - 0.7478 Validation AUC,  .9937 Train AUC - 1017 Secs

*** Conclusion: ***

From the data above its clear that higher number of n_estimators or Trees is not making much of a impact on the Validation AUC, hence we will choose:
- n_estimators=200, learning_rate=0.5 
- as its provides the optimum Validation AUC of .7374 in less amount of training time

### Ada Boost ENDS

### Gradient Boosting Classifier STARTS

loss{‘deviance’, ‘exponential’}, default=’deviance’
- loss function to be optimized. ‘deviance’ refers to deviance (= logistic regression) for classification with probabilistic outputs. For loss ‘exponential’ gradient boosting recovers the AdaBoost algorithm.

learning_ratefloat, default=0.1
- learning rate shrinks the contribution of each tree by learning_rate. There is a trade-off between learning_rate and n_estimators.

n_estimatorsint, default=100
- The number of boosting stages to perform. Gradient boosting is fairly robust to over-fitting so a large number usually results in better performance.

max_depth int, default=3
- maximum depth of the individual regression estimators. The maximum depth limits the number of nodes in the tree. Tune this parameter for best performance; the best value depends on the interaction of the input variables.




In [48]:
gradient_boosting_classifier = GradientBoostingClassifier(n_estimators=500, learning_rate=0.5,
     max_depth=4, max_leaf_nodes=15, random_state=0)

print("Gradient Boosting training started")

starting_time = time.time()

gradient_boosting_classifier.fit(x_train, y_train)

end_time = time.time()

print("Gradient Boosting training finished, took {} seconds".format(end_time - starting_time))

#Validation AUC Calculation

gradient_boosting_classifier_roc_validation = auc_calculate_print(x_validation,y_validation,gradient_boosting_classifier)


print('Validation AUC: {}'.format(gradient_boosting_classifier_roc_validation))

#Training AUC Calculation

gradient_boosting_classifier_roc_train= auc_calculate_print(x_train,y_train,gradient_boosting_classifier)


print('Train AUC: {}'.format(gradient_boosting_classifier_roc_train))

Gradient Boosting training started
Gradient Boosting training finished, took 396.37238597869873 seconds
Validation AUC: 0.7720651462620819
Train AUC: 0.8455202295730051


In [49]:
#Getting the individual Probabilities
#This is what needs to be converted to txt for submission

gradient_boosting_classifier_probs = gradient_boosting_classifier.predict_proba(Xte)
print(gradient_boosting_classifier_probs[:5])

[[0.19771194 0.80228806]
 [0.44725613 0.55274387]
 [0.80402908 0.19597092]
 [0.96567519 0.03432481]
 [0.90796277 0.09203723]]


In [50]:
# Make the prediction probabilities text file for Submission to Kaggle
SavetoTextFile('gradient_boosting',gradient_boosting_classifier_probs,Xte)

Solutions/Y_sub_gradient_boosting.txt Saved


In [51]:
#Cross Checking scores
#For reference only
y_pred = gradient_boosting_classifier.predict(x_validation)

In [52]:
#Cross Checking scores
#For reference only
cm = confusion_matrix(y_validation,y_pred)
print(cm)
accuracy_score(y_validation,y_pred)

[[23284  3007]
 [ 7446  6263]]


0.738675

*** Gradient Boosting Hyper Parameter Optimization ***


Gradient Boosting AUC At different Hyper parameters:

*** n_estimators: ***

- n_estimators=100, learning_rate=0.5,max_depth=10, max_leaf_nodes=15  - .74   Validation AUC,  .7708 
- n_estimators=200, learning_rate=0.5,max_depth=10, max_leaf_nodes=15  - .75   Validation AUC , .8059   Train AUC
- n_estimators=300, learning_rate=0.5,max_depth=10, max_leaf_nodes=15  - .7641 Validation AUC , .0.8294 Train AUC - 446 secs
- n_estimators=500, learning_rate=0.5,max_depth=10, max_leaf_nodes=15  - .7714 Validation AUC , .0.8620 Train AUC - 499 secs

*** learning_rate: ***

Changing Learning Rate = 0.1:
- n_estimators=100, learning_rate=0.1,max_depth=10, max_leaf_nodes=15  - .7194 Validation AUC , 0.7244 Train AUC - 145 secs
- n_estimators=200, learning_rate=0.1,max_depth=10, max_leaf_nodes=15  - .7321 Validation AUC , 0.7458 Train AUC - 233 secs
- n_estimators=400, learning_rate=0.1,max_depth=10, max_leaf_nodes=15  - .7474 Validation AUC , 0.7740 Train AUC - 445 secs

Changing Learning Rate = 0.2:
- n_estimators=400, learning_rate=0.2,max_depth=10, max_leaf_nodes=15  - .7631 Validation AUC , 0.8065 Train AUC - 513 secs

So, we conclude that the 0.5 learning rate seems to be the best one as its givng us .7714 Validation Accuracy with 500 estimatiors. Now checking by changing dept

*** max_depth: ***

- n_estimators=500, learning_rate=0.5, max_depth=5, max_leaf_nodes=15 - .7746 Validation AUC , 0.8550 Train AUC - 456 secs
- n_estimators=500, learning_rate=0.5, max_depth=7, max_leaf_nodes=15 - .7731 Validation AUC , 0.8598 Train AUC - 456 secs
- n_estimators=500, learning_rate=0.5, max_depth=4, max_leaf_nodes=15 - .7720 Validation AUC , 0.8455 Train AUC - 456 secs
- n_estimators=500, learning_rate=0.5, max_depth=3, max_leaf_nodes=15 - .7545 Validation AUC , 0.8455 Train AUC - 277 secs

*** Conclusion: ***

From the Above Hyper-parameter Tuning data we conclude that:

- n_estimators=500, learning_rate=0.5, max_depth=4, max_leaf_nodes=15 
- .7720 Validation AUC , 0.8455 Train AUC - 456 secs
- The above seems to be optimal as increasing the depth does not significantly improve the Validation AUC


### Gradient Boosting Classifier ENDS

### Stacking STARTS

In [53]:
#Using Dataframe's for data
# split training / validation data
X_train,X_validation,Y_train,Y_validation=model_selection.train_test_split(df_X,df_Y,test_size=0.2,random_state=42)

In [55]:
#Checking Shape
print(X_train.shape,X_validation.shape,Y_train.shape,Y_validation.shape)

(160000, 14) (40000, 14) (160000, 1) (40000, 1)


In [56]:
def Stacking(model,train,y,test,n_fold):
    folds=StratifiedKFold(n_splits=n_fold,random_state=1)
    test_pred=np.empty((0,1),float)
    train_pred=np.empty((0,1),float)
    
    for train_indices,val_indices in folds.split(train,y):
        x_train,x_val=train.iloc[train_indices,:],train.iloc[val_indices,:]
        y_train,y_val=y.iloc[train_indices],y.iloc[val_indices]
        model.fit(X=x_train,y=y_train)
        train_pred=np.append(train_pred,model.predict(x_val))
    test_pred=model.predict(test)
    
    return test_pred.reshape(-1,1),train_pred

#### Stacking For Validation Data STARTS

In [57]:
# Random Forest Stacking Model 

#For X Validation Data to check model performace

rfc_model1 = RandomForestClassifier(n_estimators=65, criterion = 'entropy', random_state = 0,n_jobs=-1, oob_score=True)

print("Random Forest Stacking training started")

starting_time = time.time()

test_X_validation_pred1 ,train_X_validation_pred1=Stacking(model=rfc_model1,n_fold=10, train=X_train,test=X_validation,y=Y_train)

end_time = time.time()
print("Random Forest Stacking training finished, took {} seconds".format(end_time - starting_time))


train_X_validation_pred1=pd.DataFrame(train_X_validation_pred1)
test_X_validation_pred1=pd.DataFrame(test_X_validation_pred1)

Random Forest Stacking training started
Random Forest Stacking training finished, took 487.4433162212372 seconds


In [59]:
#Ada Boost Stacking Model 2 
#For X Validation Data to check model performace

ada_boost_model2 =AdaBoostClassifier(tree.DecisionTreeClassifier(random_state=0),algorithm='SAMME.R',
                                     n_estimators=200, learning_rate=0.5)

print("Ada Boost Stacking training started")

starting_time = time.time()

test_X_validation_pred2 ,train_X_validation_pred2=Stacking(model=ada_boost_model2,n_fold=10, 
                                                           train=X_train,test=X_validation,y=Y_train)

end_time = time.time()
print("Ada Boost Stacking training finished, took {} seconds".format(end_time - starting_time))

train_X_validation_pred2=pd.DataFrame(train_X_validation_pred2)

test_X_validation_pred2=pd.DataFrame(test_X_validation_pred2)

Ada Boost Stacking training started
Ada Boost Stacking training finished, took 7819.5468628406525 seconds


In [60]:
#Gradient Boosting Stacking Model 3
#For X Validation Data to check model performace

gradient_boost_model3 =gradient_boosting_classifier = GradientBoostingClassifier(n_estimators=500, learning_rate=0.5,
     max_depth=4, max_leaf_nodes=15, random_state=0)

print("Gradient Boosting Stacking training started (Validation Data)")

starting_time = time.time()

#XValidation data
test_X_validation_pred3 ,train_X_validation_pred3=Stacking(model=gradient_boost_model3,n_fold=10, train=X_train,
                                                           test=X_validation,y=Y_train)

end_time = time.time()

print("Gradient Boosting Stacking training finished, took {} seconds".format(end_time - starting_time))

train_X_validation_pred3=pd.DataFrame(train_X_validation_pred3)

test_X_validation_pred3=pd.DataFrame(test_X_validation_pred3)

Gradient Boosting Stacking training started (Validation Data)
Gradient Boosting Stacking training finished, took 2597.1103184223175 seconds


#### Combining All 3 Models Random Forest, AdaBoost, Gradient Boosting STARTS

In [61]:
# For X_Validation Data

validation_prediction = pd.concat([train_X_validation_pred1, train_X_validation_pred2,train_X_validation_pred3], axis=1)

test_prediction = pd.concat([test_X_validation_pred1, test_X_validation_pred2, test_X_validation_pred3], axis=1)

model = LogisticRegression(random_state=1)

#Fitting model on Train Data
model.fit(validation_prediction,Y_train)

#Calculating Score on Test Prediction & Y_validation(this is our true test data for validation)

model.score(test_prediction, Y_validation) 

0.737075

In [62]:
#Getting the individual Probabilities

probs = model.predict_proba(test_prediction)

In [63]:
#Execute for X_Validation ONLY
#ROC / AUC Score

# Stacking_LogisticRegression_classifier_roc = metrics.roc_auc_score(
#     Y_validation, model.predict_proba(test_prediction)[:,1])

Stacking_LogisticRegression_classifier_roc_validation = auc_calculate_print(test_prediction,Y_validation,model)


print('Stacked Logistic Regression Validation AUC: {}'.format(Stacking_LogisticRegression_classifier_roc_validation))

#Training AUC Calculation

# Stacking_LogisticRegression_classifier_roc_train = metrics.roc_auc_score(
#     Y_train, model.predict_proba(validation_prediction)[:,1])

Stacking_LogisticRegression_classifier_roc_train = auc_calculate_print(validation_prediction,Y_train,model)

print('Stacked Logistic Regression Train AUC: {}'.format(Stacking_LogisticRegression_classifier_roc_train))

Stacked Logistic Regression Validation AUC: 0.7266510009026358
Stacked Logistic Regression Train AUC: 0.7249642711096788


#### Combining All 3 Models Random Forest, AdaBoost, Gradient Boosting ENDS

#### Stacking For Validation Data ENDS

#### Stacking For Xte(Kaggle Submission) Data STARTS

In [ ]:
#Random Forest Model 1
#Kaggle Test data

rfc_model1 = RandomForestClassifier(n_estimators=65, criterion = 'entropy', random_state = 0,n_jobs=-1, oob_score=True)

print("Random Forest Kaggle Test Data training started")
starting_time = time.time()

test_pred1 ,train_pred1=Stacking(model=rfc_model1,n_fold=10, train=X_train,test=df_Xte,y=Y_train)

end_time = time.time()
print("Random Forest Kaggle Test Data training finished, took {} seconds".format(end_time - starting_time))

train_pred1=pd.DataFrame(train_pred1)
test_pred1=pd.DataFrame(test_pred1)

In [ ]:
#Ada Boost Model 2
#Use this KAGGLE Test dataset

ada_boost_model2 =AdaBoostClassifier(tree.DecisionTreeClassifier(random_state=0),algorithm='SAMME.R',
                                     n_estimators=200, learning_rate=0.5)

#Actual Test Data to be submitted to Kaggle
test_pred2 ,train_pred2=Stacking(model=ada_boost_model2,n_fold=10, train=X_train,test=df_Xte,y=Y_train)

train_pred2=pd.DataFrame(train_pred2)

test_pred2=pd.DataFrame(test_pred2)

In [ ]:
#Gradient Boosting Model 3
#Use this KAGGLE Test dataset

gradient_boost_model3 = GradientBoostingClassifier(n_estimators=500, learning_rate=0.5,
     max_depth=4, max_leaf_nodes=15, random_state=0)

print("Gradient Boosting  Kaggle Test Data training started")
starting_time = time.time()

#Actual Test Data to be submitted to Kaggle
test_pred3 ,train_pred3=Stacking(model=gradient_boost_model3,n_fold=10, train=X_train,test=df_Xte,y=Y_train)

end_time = time.time()
print("Gradient Boosting  Kaggle Test Data training finished, took {} seconds".format(end_time - starting_time))

train_pred3=pd.DataFrame(train_pred3)

test_pred3=pd.DataFrame(test_pred3)

#### Kaggle Test Data - Combining All 3 Models Random Forest, AdaBoost, Gradient Boosting STARTS

In [ ]:
validation_prediction = pd.concat([train_pred1, train_pred2,train_pred3], axis=1)
test_prediction = pd.concat([test_pred1, test_pred2, test_pred3], axis=1)

model = LogisticRegression(random_state=1)

model.fit(validation_prediction,Y_train)

#You we will not be able to score as this scoring is done by Kaggle; as we dont have y_validation / y_test 

# model.score(test_prediction, y_test)
# model.score(test_prediction, Y_validation) # # We have to evaluate this in Kaggle as we dont have the df_Yte values

In [ ]:
#Getting the individual Probabilities
#This is what needs to be converted to txt for submission
probs = model.predict_proba(test_prediction)

In [ ]:
#Execute for Kaggle Test Data
#Writing Probabilities to file

SavetoTextFile('stacked_model',probs,df_Xte)

# Create the data for submission by taking the P(Y=1) column from probs and just add a running index as the first column.
# Y_sub = np.vstack([np.arange(Xte.shape[0]), probs[:, 1]]).T
# Y_sub = np.vstack([np.arange(df_Xte.shape[0]), probs[:, 1]]).T  #For df_Xte

# We specify the header (ID, Prob1) and also specify the comments as '' so the header won't be commented out with
# the # sign.
# np.savetxt('/content/shangyihuang/data/Y_sub_June_4_2020.txt', Y_sub, '%d, %.5f', header='ID,Prob1', comments='', delimiter=',')

#### Kaggle Test Data - Combining All 3 Models Random Forest, AdaBoost, Gradient Boosting ENDS

#### Stacking For Xte(Kaggle Submission) Data ENDS

### Stacking ENDS